In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import networkx as nx

#import sys
#sys.path.append(r"C:\Users\salva\PycharmProjects\SpreadPy")
#path = r"C:\Users\salva\PycharmProjects\SpreadPy\\"

from SpreadPy import IOUtils
from SpreadPy import AnalyticalUtils
from SpreadPy.Models.models import *

import matplotlib.pyplot as plt
%matplotlib inline

## Build the Multiplex Graph as a Networkx Object

In [ ]:
io = IOUtils()

#folder = path+'toy_data/mental_lexicon_2/'
folder = 'toy_data/mental_lexicon_2/'
g_multiplex, layer_names = io.read_example_mental_lexicon(folder)

In [ ]:
layer_names

In [ ]:
'N', g_multiplex.number_of_nodes(), 'E', g_multiplex.number_of_edges()

In [ ]:
g_multiplex.edges['world','globe']

In [ ]:
g_multiplex.edges['alone','lone']['layer']

# Use the layer(s) you prefer

In [ ]:
utils = AnalyticalUtils(g_multiplex)

In [ ]:
g_sem = utils.graph_for_layers(layer_names=['FreeAssociations'])
'N', g_sem.number_of_nodes(), 'E', g_sem.number_of_edges()

In [ ]:
largest_component = max(nx.connected_components(g_sem), key=len)
lcc_sem = g_sem.subgraph(largest_component)
lcc_sem = lcc_sem.copy()

'N', lcc_sem.number_of_nodes(), 'E', lcc_sem.number_of_edges()

In [ ]:
g_phon = utils.graph_for_layers(layer_names=['PhonSimilarities'])
'N', g_phon.number_of_nodes(), 'E', g_phon.number_of_edges()

In [ ]:
largest_component = max(nx.connected_components(g_phon), key=len)
lcc_phon = g_phon.subgraph(largest_component)
lcc_phon = lcc_phon.copy()

'N', lcc_phon.number_of_nodes(), 'E', lcc_phon.number_of_edges()

# Largest Viable Component

In [ ]:
lvc_nodes = utils.lvc(layer_names)
len(lvc_nodes)

In [ ]:
to_rem = list(set(g_multiplex.nodes())^set(lvc_nodes))
g_multiplex.remove_nodes_from(to_rem)

'N', g_multiplex.number_of_nodes(), 'E', g_multiplex.number_of_edges()

In [ ]:
g_sem = utils.graph_for_layers(layer_names=['FreeAssociations'])
'N', g_sem.number_of_nodes(), 'E', g_sem.number_of_edges()

In [ ]:
#g_phon = utils.graph_for_layers(layer_names=['phonology'])
g_phon = utils.graph_for_layers(layer_names=['PhonSimilarities'])
'N', g_phon.number_of_nodes(), 'E', g_phon.number_of_edges()

# Adding Node Attributes

I will use the semantic network as an example.

I build a dictionary in the following form:

key_1 -> {key_2: value}

key_1 is the name of the attribute, e.g., 'concreteness', key_2 is the node id and value is the value of the attribute, e.g., {'rock': 7, 'sand': 5}

In [ ]:
# if it does not work, run:

# import nltk
# nltk.download('wordnet')


# here you can use any attribute you want (freq, concr, etc)
additional_attrs = {'lorem': {n: 1 for n in g_sem.nodes()},
                   'ipsum': {n: 'A' for n in g_sem.nodes()}
                   }


utils = AnalyticalUtils(g_sem)
utils.word_features_pipeline(binned=False, additional_attributes=additional_attrs)

In [ ]:
g_sem.nodes[('mom')]

# Weighted Spreading Activation: Similarity Model

### Toy similarity model

<b> weighted=True </b> : I will consider weights, i.e., node attributes;

<b> weight='toy' </b> : The name of the node attribute to consider;

<b> similarity = True </b> : The flow will be higher between nodes sharing similar values;

In [ ]:
model = BaseSpreading(g_sem, retention=0.7, decay=0.1, suppress=0,
                         weighted=True, weight='length', similarity=True)

################### INITIAL SETTING ##############
initial_active_nodes = ['mom']
initial_status = {node: g_sem.degree[node] if node in initial_active_nodes else 0 for node in g_sem.nodes}
model.status = initial_status
time = 20
##################################################

results = model.iteration_bunch(time)
final_status = results[-1]['status']
ranked_results = model.rank_status_activation(final_status)

In [ ]:
node_activation = model.node_activation_series(results)

In [ ]:
model.plot_activated_series(node_activation, time, ['dad', 'ma', 'maternal'])

# Weighted Spreading Activation: Dissimilarity Model

### Toy dissimilarity model

<b> weighted=True </b> : I will consider weights, i.e., node attributes;

<b> weight='toy' </b> : The name of the node attribute to consider;

<b> similarity = False </b> : The flow will be higher between nodes sharing different values;

In [ ]:
model = BaseSpreading(g_sem, retention=0.7, decay=0.1, suppress=0,
                         weighted=True, weight='length', similarity=False)

################### INITIAL SETTING ##############
initial_active_nodes = ['mom']
initial_status = {node: g_sem.degree[node] if node in initial_active_nodes else 0 for node in g_sem.nodes}
model.status = initial_status
time = 20
##################################################

results = model.iteration_bunch(time)
final_status = results[-1]['status']
ranked_results = model.rank_status_activation(final_status)

In [ ]:
node_activation = model.node_activation_series(results)

In [ ]:
model.plot_activated_series(node_activation, time, ['dad', 'ma', 'maternal'])